# Library

In [1]:
import pandas as pd
from pmaw import PushshiftAPI
api = PushshiftAPI()
import datetime as dt
import numpy as np
import praw
import time
#from redditbots_function import flatten_author, author_comments

# Set up

In [2]:
# Praw Enhancement
reddit = praw.Reddit(
 client_id='9wXh5oa4cfW07_eUn5Hu3A',
 client_secret='m3GnhaEvbM5LGCWX3BMghLCatOLN3g',
 user_agent=f'python: PMAW request enrichment (by u/softyarn)'
)
api_praw = PushshiftAPI(praw=reddit)
# Pmaw
api = PushshiftAPI(max_sleep = 60, shards_down_behavior = None)

# Function

In [3]:
def get_epochdate(bot_epoch, duration):
    before = int((dt.datetime.fromtimestamp(bot_epoch)+dt.timedelta(days = duration)).timestamp())
    after = int((dt.datetime.fromtimestamp(bot_epoch)-dt.timedelta(days = duration)).timestamp())
    return before, after
def within60days(df, bot_epoch, duration):
    before, after = get_epochdate(bot_epoch = bot_epoch, duration = duration)
    res = df.loc[(df['created_utc'] >= after) & (df['created_utc'] <= before)]
    return res
def author_list(df):
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    authors = nomod.loc[:,'author']
    authors_unique = np.unique(authors)
    return authors_unique
def author_comments(df, bot_epoch, duration, limit = 10000000000):
    before, after = get_epochdate(bot_epoch, duration) 
    #creating loop so that only 100 authors per praw
    tic = time.time()
    global res
    res = []
    authors = author_list(df)
    i = 0
    while i < len(authors):
        j = min(i+2, len(authors))
        x = ','.join(authors[i:j])
        comments = api.search_comments(author = x, limit = limit, before = before, after = after, filter = ['id', 'banned_at_utc', 'mod_reason_title', 'author', 'created_utc', 'parent_id', 'subreddit_id', 'body'], mem_safe= True)
        res += [c for c in comments]
        if i%500 == 0:
            res_out = pd.DataFrame(res)
            res_out.to_csv('author_comments_' + str(i/1000) + '.csv', encoding = 'utf_8_sig')
            res = []
        i +=2
        if i%100 == 0:
            time.sleep(1)
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i+5)
    res_out = pd.DataFrame(res)
    res_out.to_csv('author_comments_Finale.csv', encoding = 'utf_8_sig')   
    return res_out

# Running

In [4]:
fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")

C:\Users\nguye\AppData\Local\Temp\ipykernel_33000\3957158027.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")


In [5]:
fds1 = within60days(fds, bot_epoch = 1572234835 , duration = 30)

In [ ]:
len(author_list(fds1))

In [6]:
fds2 = fds1[4400:4774]

In [7]:
df_res = author_comments(df = fds2, bot_epoch = 1572234835, duration = 30, limit = None)

Time elapsed:  713  secs, i =  105
